# CONOCIMIENTO DE LOS DATOS

## Planteamiento del problema

Bob ha fundado su propia empresa de telefonía móvil y busca competir con gigantes del mercado como Apple, Samsung, etc. Para abordar este desafío, se han recopilado datos de ventas de teléfonos móviles de diversas empresas y se busca identificar patrones y relaciones entre las características técnicas de los dispositivos.

En este contexto altamente competitivo, estimar el precio de venta de los teléfonos móviles es crucial para el éxito de su empresa.

¿Cuál es el mejor modelo predictivo para estimar los rangos de precios de teléfonos móviles, basado en características técnicas?

El objetivo de este proyecto, es desarrollar un modelo predictivo que pueda asignar a cada teléfono móvil un rango de precios, para proporcionar una guía clara sobre cómo posicionar los productos en el mercado. Esta predicción de rango de precios ayudará a Bob, a tomar decisiones estratégicas fundamentadas en datos, optimizando así sus estrategias de precios y su competitividad en el mercado de telefonía móvil.

## Variables del Dataset
- battery_power: Energía total que una batería puede almacenar en un tiempo medida en mAh
- blue: Tiene bluetooth o no
- clock_speed: Velocidad a la que el microprocesador ejecuta instrucciones
- dual_sim: Tiene soporte dual sim o no
- fc: Megapíxeles de la cámara frontal
- four_g: Tiene 4G o no
- int_memory: Memoria interna en Gigabytes
- m_dep: Profundidad del móvil en cm
- mobile_wt: Peso del teléfono móvil
- n_cores: Número de núcleos de procesador
- pc: Megapíxeles de la cámara principal
- px_height: Altura de resolución de píxeles
- px_width: Ancho de resolución de píxeles
- ram: Memoria de acceso aleatorio en megabytes
- sc_h: Altura de pantalla del móvil en cm
- sc_w: Ancho de pantalla del móvil en cm
- talk_time: mayor tiempo que durará una sola carga de batería cuando estés en llamada
- three_g: Tiene 3G o no
- touch_screen: Tiene pantalla táctil o no
- wifi: Tiene wifi o no
- price_range: Esta es la variable objetivo con valor de 0 (bajo costo), 1 (costo medio), 2 (costo alto) y 3 (costo muy alto).

Dataset "train" lo llamaremos "df_mobile_market" que hace referencia a los datos de ventas de teléfonos móviles, que Bob a recopilado, de diversas empresas

Dataset "test" lo llamaremos "df_feature_mobile" que hace referencia a los todos los telefonos a los cuales se necesita asignar un rango de precio

## Importar librerías

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import StratifiedShuffleSplit, GridSearchCV, cross_val_score, cross_val_predict
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_selection import SelectKBest, f_classif, RFECV
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, confusion_matrix
import warnings

In [2]:
# Desactivar las alertas

warnings.filterwarnings('ignore')

In [3]:
# Mostar todas las columnas de los DataFrame

pd.set_option('display.max_columns', None)

## Importar datos

In [6]:
# Importar las datasets

df_mobile_market = pd.read_csv(r'C:\Users\nuria\OneDrive\Escritorio\Data Science\Fundamentals\online_ds_thebridge_nuria\Analisis_de_datos\ML\Precios_Telefonia\train (1).csv')
df_feature_mobile = pd.read_csv(r'C:\Users\nuria\OneDrive\Escritorio\Data Science\Fundamentals\online_ds_thebridge_nuria\Analisis_de_datos\ML\Precios_Telefonia\test (1).csv', index_col = 'id')

# Visualizar DataFrame df_mobile_market
df_mobile_market.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi,price_range
0,842,0,2.2,0,1,0,7,0.6,188,2,2,20,756,2549,9,7,19,0,0,1,1
1,1021,1,0.5,1,0,1,53,0.7,136,3,6,905,1988,2631,17,3,7,1,1,0,2
2,563,1,0.5,1,2,1,41,0.9,145,5,6,1263,1716,2603,11,2,9,1,1,0,2
3,615,1,2.5,0,0,0,10,0.8,131,6,9,1216,1786,2769,16,8,11,1,0,0,2
4,1821,1,1.2,0,13,1,44,0.6,141,2,14,1208,1212,1411,8,2,15,1,1,0,1


In [7]:
# Visualizar DataFrame df_feature_mobile
df_feature_mobile.head()

,battery_power,blue,clock_speed,dual_sim,fc,four_g,int_memory,m_dep,mobile_wt,n_cores,pc,px_height,px_width,ram,sc_h,sc_w,talk_time,three_g,touch_screen,wifi
id,,,,,,,,,,,,,,,,,,,,
1,1043,1,1.8,1,14,0,5,0.1,193,3,16,226,1412,3476,12,7,2,0,1,0
2,841,1,0.5,1,4,1,61,0.8,191,5,12,746,857,3895,6,0,7,1,0,0
3,1807,1,2.8,0,1,0,27,0.9,186,3,4,1270,1366,2396,17,10,10,0,1,1
4,1546,0,0.5,1,18,1,25,0.5,96,8,20,295,1752,3893,10,0,7,1,1,0
5,1434,0,1.4,0,11,1,49,0.5,108,6,18,749,810,1773,15,8,7,1,0,1


## ANÁLISI EXPLORATORIO DE DATOS

In [8]:
df_mobile_market.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2000 entries, 0 to 1999
Data columns (total 21 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  2000 non-null   int64  
 1   blue           2000 non-null   int64  
 2   clock_speed    2000 non-null   float64
 3   dual_sim       2000 non-null   int64  
 4   fc             2000 non-null   int64  
 5   four_g         2000 non-null   int64  
 6   int_memory     2000 non-null   int64  
 7   m_dep          2000 non-null   float64
 8   mobile_wt      2000 non-null   int64  
 9   n_cores        2000 non-null   int64  
 10  pc             2000 non-null   int64  
 11  px_height      2000 non-null   int64  
 12  px_width       2000 non-null   int64  
 13  ram            2000 non-null   int64  
 14  sc_h           2000 non-null   int64  
 15  sc_w           2000 non-null   int64  
 16  talk_time      2000 non-null   int64  
 17  three_g        2000 non-null   int64  
 18  touch_sc

In [9]:
df_feature_mobile.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1000 entries, 1 to 1000
Data columns (total 20 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   battery_power  1000 non-null   int64  
 1   blue           1000 non-null   int64  
 2   clock_speed    1000 non-null   float64
 3   dual_sim       1000 non-null   int64  
 4   fc             1000 non-null   int64  
 5   four_g         1000 non-null   int64  
 6   int_memory     1000 non-null   int64  
 7   m_dep          1000 non-null   float64
 8   mobile_wt      1000 non-null   int64  
 9   n_cores        1000 non-null   int64  
 10  pc             1000 non-null   int64  
 11  px_height      1000 non-null   int64  
 12  px_width       1000 non-null   int64  
 13  ram            1000 non-null   int64  
 14  sc_h           1000 non-null   int64  
 15  sc_w           1000 non-null   int64  
 16  talk_time      1000 non-null   int64  
 17  three_g        1000 non-null   int64  
 18  touch_screen 

In [10]:
# Visualizar datos en blanco del DataFrame df_mobile_market

datos_blanco = pd.DataFrame({
    'Registros en Blanco' : df_mobile_market.isna().sum(),
    '% Registros en Blanco' : df_mobile_market.isna().sum()/len(df_mobile_market)
})

datos_blanco

,Registros en Blanco,% Registros en Blanco
battery_power,0,0.0
blue,0,0.0
clock_speed,0,0.0
dual_sim,0,0.0
fc,0,0.0
four_g,0,0.0
int_memory,0,0.0
m_dep,0,0.0
mobile_wt,0,0.0
n_cores,0,0.0
